In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Loading Bitcoin USA Dataset
data = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2018-01-08.csv')
#Looking at The data Set
data.head()


In [ ]:
#Taking in new smaple set
#len(data)
X = data.iloc[2998500:3160999,1:7].values
Y = data.iloc[2998500:3160999,7:].values



In [ ]:
#Creating A Minutes stamp for the RNN
# 20/10 Minutes is the most Idea for This
Minutes = 180
next_Minutes = 5
X_train = []
y_train = []

In [ ]:
#Data is Being Scaled to 0-1
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_X_scaled = sc.fit_transform(X)

sc_Y = MinMaxScaler(feature_range = (0, 1))
testing_y_scaled = sc_Y.fit_transform(Y)


In [ ]:
#Splitting Data with 0 to  Minutes data
#Then splitting into Test set and Train set
#Notes data set contain High Low , Open and close
for i in tqdm(range(Minutes, len(X)-next_Minutes)):
    X_train.append(training_X_scaled[i-Minutes:i,:])
    y_train.append(testing_y_scaled[i:i+next_Minutes, 0])
X, y = np.array(X_train), np.array(y_train)
X_train, y_train = X[0:len(X)-500],y[0:len(X)-500]
X_test, y_test = X[len(X)-500:len(X)],y[len(X)-500:len(X)]


In [ ]:
#Importing ML Lib
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import load_model




In [ ]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding a Third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding a Fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a Final LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = next_Minutes))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor = load_model("../input/model-bitcoin/Final_model.hd5")


In [ ]:
regressor.fit(X_train, y_train, epochs = 2, batch_size = 100)

In [ ]:
#Predicting With Our test set 
regressor.save('Final_model.hd5')
pred_scores = regressor.predict(X_test)


In [ ]:
contents = []
start_range = 150
end_range   = start_range+next_Minutes
for i in range(next_Minutes):
    contents.append(pred_scores[start_range,i])



In [ ]:
# Visualising the results (Full Plot)

import matplotlib.pyplot as plt
plt.figure(None,figsize = (24,12))
plt.plot(y_test[:,0], color = 'red', label = 'Real Price of Bitcoin')
plt.plot(pred_scores[next_Minutes:len(y_test)-next_Minutes,next_Minutes-1:next_Minutes], color = 'blue', label = 'Predicted Price of Bitcoin')
plt.title('Prediction  graph')
plt.xlabel('Time(Minutes)')
plt.ylabel('Price(IN BT)')
plt.legend()
plt.show()


In [ ]:
# A portion of the Plot
plt.figure(None,figsize = (24,12))
plt.plot(y_test[start_range:end_range,1], color = 'red', label = 'Real Price of Bitcoin')
plt.plot(contents, color = 'blue', label = 'Predicted Price of Bitcoin')
plt.title('Prediction  graph')
plt.xlabel('Time(Minutes)')
plt.ylabel('Price(IN BT)')
plt.legend()
plt.show()

In [ ]:
'''My firt take On RNN, I know there is a lot of room to improve . This is the start'''